## 2.1 Stereo Calibration with pixelization

### import  libraries

In [2]:
vscode = 1

In [3]:
if(vscode == 1):
    # for vscode
    %matplotlib qt
else:
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
DECIMALS = 2  # how many decimal places to use in print


In [5]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,4])                     # camera centre

C_R = np.array([500,0,4])

chess_dimx,chess_dimy = (8,6)
chess_sq_size = 20


In [6]:
world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
X = get_chessboard_rot_trans(world_pts,rx = THETA_X,ry = THETA_Y,rz = THETA_Z,tx = 250,ty = 0,tz  = 800)

In [7]:
def create_matrix_A(hom_points_3d, hom_points_2d):
    """build matrix A for DLT method"""

    assert hom_points_3d.ndim == 2 and hom_points_3d.shape[1] == 4
    assert hom_points_2d.ndim == 2 and hom_points_2d.shape[1] == 3
    assert hom_points_3d.shape[0] == hom_points_2d.shape[0]

    N = hom_points_3d.shape[0]

    _A = []
    for i in range(N):
        X, Y, Z, S = hom_points_3d[i]
        u, v, w = hom_points_2d[i]

        _A.append([  0,   0,   0,   0, -w*X, -w*Y, -w*Z, -w*S,  v*X,  v*Y,  v*Z,  v*S])
        _A.append([w*X, w*Y, w*Z, w*S,    0,    0,    0,    0, -u*X, -u*Y, -u*Z, -u*S])

    A = np.array(_A, dtype=np.float64)
    assert A.shape == (2*N, 12)
    return A


def get_normalize_2d_matrix(points_2d):
    """normalize 2d points to mean 0 and rms sqrt(2)"""

    pts_mean = points_2d.mean(axis=0)
    centered_pts_2d = points_2d - pts_mean
    s = 1 / np.linalg.norm(centered_pts_2d).mean()
    xm, ym = pts_mean
    T = np.array([[s, 0, -s*xm],
                     [0, s, -s*ym],
                     [0, 0,  1 ]], dtype=np.float64)
    return T


def get_normalize_3d_matrix(points_3d):
    """normalize 3d points to mean 0 and rms sqrt(2)"""

    pts_mean = points_3d.mean(axis=0)
    centered_pts_3d = points_3d - pts_mean
    s = 1 / np.linalg.norm(centered_pts_3d).mean()
    xm, ym, zm = pts_mean
    U = np.array([[s, 0, 0, -s*xm],
                     [0, s, 0, -s*ym],
                     [0, 0, s, -s*zm],
                     [0, 0, 0,  1 ]], dtype=np.float64)
    return U


def get_homogeneous_coordinates(points):
    """return points in homogeneous coordinates"""

    assert points.ndim == 2
    assert points.shape[1] in [2, 3]
    if points.shape[1] == 3:
        assert not np.allclose(points[:,2], 1.)
    return np.hstack((points, np.ones((points.shape[0], 1))))


def project_3d_to_pixel(P,pts3d, distorted=True):
    pts3d = np.array(pts3d,copy=False)
    assert pts3d.ndim==2
    assert pts3d.shape[1]==3

    # homogeneous and transposed
    pts3d_h = np.empty( (4,pts3d.shape[0]) )
    pts3d_h[:3,:] = pts3d.T
    pts3d_h[3] = 1

    M = P

    # undistorted homogeneous image coords
    cc = np.dot(M, pts3d_h)

    # project
    pc = cc[:2]/cc[2]
    u, v = pc

    return np.vstack((u,v)).T

def DLT_test(X3d, x2d, width=640, height=480):
    """Given 3D coordinates X3d and 2d coordinates x2d, find camera model"""

    # Note: may want to implement Hatze (1988). This returns
    # orthogonal rotation matrix. See notes and implementation in
    # https://github.com/NatPRoach/HawkMothCode/blob/master/ManualExtractionCode/DLTcal5.m

    # Implementation of DLT algorithm from
    # "Hartley & Zisserman - Multiple View Geometry in computer vision - 2nd Edition"

    # Normalize 2d points and keep transformation matrix
    T = get_normalize_2d_matrix(x2d)
    Tinv = np.linalg.inv(T)
    hom_points_2d = get_homogeneous_coordinates(x2d)
    normalized_points_2d = np.empty(hom_points_2d.shape)
    for i, x in enumerate(hom_points_2d):
        normalized_points_2d[i,:] = np.dot(T, x)

    # Normalize 3d points and keep transformation matrix
    U = get_normalize_3d_matrix(X3d)
    hom_points_3d = get_homogeneous_coordinates(X3d)
    normalized_points_3d = np.empty(hom_points_3d.shape)
    for i, x in enumerate(hom_points_3d):
        normalized_points_3d[i,:] = np.dot(U, x)

    # get matrix A
    A = create_matrix_A(normalized_points_3d, normalized_points_2d)

    # solve via singular value decomposition
    _, singular_values, VT = np.linalg.svd(A, full_matrices=False)
    sol_idx = np.argmin(singular_values)
    assert sol_idx == 11
    Pvec_n = VT.T[:,sol_idx]  # that's why we need to pick the rows here...

    P_n = Pvec_n.reshape((3, 4))

    # Denormalize
    P = np.dot(Tinv, np.dot(P_n, U))
    P = P / P[-1, -2]
  
    return P,normalized_points_2d,normalized_points_3d


In [8]:
x_arr_L, X_arr_L, E_L,K_L = get_image_points(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F*(1/pixel_width))
X_h_L = to_homogeneus_arr(X_arr_L)
x_h_L = to_homogeneus_arr(x_arr_L)

P_L = DLT(x_h_L,X_h_L)

print("Camera Matrix:\n",P_L)

n_points = X_h_L.shape[0]
xEst_L = np.zeros((n_points,2))

for j in range(n_points):
    X_pts = to_homogeneous(X_arr_L[j])
    x_aux  = np.dot(P_L,X_pts)
    xEst_L[j,:] = to_inhomogeneous(x_aux)
    
err_list_L = np.mean(np.sqrt(np.sum(np.square(x_arr_L-xEst_L), 1)))

print("Geometric error:",err_list_L)

Camera Matrix:
 [[-4.40418826e+02 -1.32894809e-09 -1.46272586e+02 -7.27071403e+03]
 [-2.26703184e-10 -4.40418826e+02  2.39044089e+03 -1.99041840e+06]
 [-3.82206426e-13  8.09261221e-14  1.00000000e+00 -9.28397503e+02]]
Geometric error: 4.298524065612823e-10


In [9]:
x_arr_L, X_arr_L, E_L,K_L = get_image_points(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F*(1/pixel_width))
#print(x_arr_L)
# to pixels
#x_arr_L_pixel = x_arr_L/pixel_width

X_h_L = to_homogeneus_arr(X_arr_L)
x_h_L = to_homogeneus_arr(x_arr_L)

Txyz, X_h_L_n = normalization(X_arr_L, 2)
Txy, x_h_L_n = normalization(x_h_L, 2)


P_n_L = DLT(x_h_L,X_h_L)

print("Camera Matrix:\n",P_n_L)

n_points = X_h_L.shape[0]
xEst_L = np.zeros((n_points,2))

for j in range(n_points):
    X_pts = to_homogeneous(X_arr_L[j])
    x_aux  = np.dot(P_n_L,X_pts)
    xEst_L[j,:] = to_inhomogeneous(x_aux)
    
err_list_L = np.mean(np.sqrt(np.sum(np.square(x_arr_L-xEst_L), 1)))

print("Geometric error:",err_list_L)

Camera Matrix:
 [[-4.40418826e+02 -1.32894809e-09 -1.46272586e+02 -7.27071403e+03]
 [-2.26703184e-10 -4.40418826e+02  2.39044089e+03 -1.99041840e+06]
 [-3.82206426e-13  8.09261221e-14  1.00000000e+00 -9.28397503e+02]]
Geometric error: 4.298524065612823e-10


In [10]:
x_arr_R, X_arr_R, E_R,K_R = get_image_points(X,PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F*(1/pixel_width))

X_h_R = to_homogeneus_arr(X_arr_R)
x_h_R = to_homogeneus_arr(x_arr_R)


P_n_R = DLT(x_h_R,X_h_R)


print("Camera Matrix:\n",P_n_R)

n_points = X_h_R.shape[0]
xEst_R = np.zeros((n_points,2))

for j in range(n_points):
    X_pts = to_homogeneous(X_arr_R[j])
    x_aux  = np.dot(P_n_R,X_pts)
    xEst_R[j,:] = to_inhomogeneous(x_aux)
    
err_list_R = np.mean(np.sum(np.square(x_arr_R-xEst_R), 1))

print("Geometric error:",err_list_R)

Camera Matrix:
 [[ 2.81542963e+04  3.60982294e-09 -7.65924720e+03 -4.44242440e+03]
 [-3.68523887e-10  2.81542963e+04 -1.55894977e+05  1.29706423e+08]
 [ 4.77229130e-13  1.42825409e-11  1.00000000e+00  7.40796277e+03]]
Geometric error: 1.2565361055780401e-21


In [11]:
fig = plt.figure(figsize=(20,11))
fig.suptitle('Left and Right image coordinates', fontsize=16)
ax_L = fig.add_subplot(121)
ax_L.plot(x_arr_L[:,0],x_arr_L[:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_L.set_title("reprojection error: " + str(err_list_L))
ax_L.set_xlabel("X axis")
ax_L.set_ylabel("Y axis")
ax_L.grid()
ax_L.plot(xEst_L[:,0],xEst_L[:,1], color = 'b',ls = "None", marker = "*", label = 'Estimation')
ax_L.legend()
ax_L.set_xlim([0,image_size[0]])
ax_L.set_ylim([0,image_size[1]])


ax_R = fig.add_subplot(122)
ax_R.plot(x_arr_R[:,0],x_arr_R[:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_R.set_title("reprojection error: " + str(err_list_R))
ax_R.set_xlabel("X axis")
ax_R.set_ylabel("Y axis")
ax_R.grid()
ax_R.plot(xEst_R[:,0],xEst_R[:,1], color = 'b',ls = "None", marker = "*", label = 'Estimation')
ax_R.legend()
ax_R.set_xlim([0,image_size[0]])
ax_R.set_ylim([0,image_size[1]])
plt.show()

In [12]:
world_origin = np.zeros(3)
dx, dy, dz = np.eye(3)

world_frame = ReferenceFrame(
    origin=world_origin, 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)
camera_frameL = ReferenceFrame(
    origin=C_L, 
    dx=E_L[:,:3] @ dx, 
    dy=E_L[:,:3] @ dy,
    dz=E_L[:,:3] @ dz,
    name="Camera 1 ",
)
camera_frameR = ReferenceFrame(
    origin=C_R, 
    dx=E_R[:,:3] @ dx, 
    dy=E_R[:,:3] @ dy,
    dz=E_R[:,:3] @ dz,
    name="Camera 2",
)

Z_L = PrincipalAxis(
    camera_center=camera_frameL.origin,
    camera_dz=camera_frameL.dz,
    f=F,
)
Z_R = PrincipalAxis(
    camera_center=camera_frameR.origin,
    camera_dz=camera_frameR.dz,
    f=F,
)
image_frameL = ReferenceFrame(
    origin=Z_L.p - camera_frameL.dx * (PX*pixel_width) - camera_frameL.dy * (PY*pixel_width), 
    dx=E_L[:,:3] @ dx, 
    dy=E_L[:,:3] @ dy,
    dz=E_L[:,:3] @ dz,
    name="Image",
)
image_frameR = ReferenceFrame(
    origin=Z_R.p - camera_frameR.dx * (PX*pixel_width) - camera_frameR.dy * (PY*pixel_width), 
    dx=E_R[:,:3] @ dx, 
    dy=E_R[:,:3] @ dy,
    dz=E_R[:,:3] @ dz,
    name="Image",
)
image_planeL = ImagePlane(
    origin=image_frameL.origin, 
    dx=image_frameL.dx, 
    dy=image_frameL.dy, 
    heigth=IMAGE_HEIGTH*pixel_width,
    width=IMAGE_WIDTH*pixel_width,
)
image_planeR = ImagePlane(
    origin=image_frameR.origin, 
    dx=image_frameR.dx, 
    dy=image_frameR.dy, 
    heigth=IMAGE_HEIGTH*pixel_width,
    width=IMAGE_WIDTH*pixel_width,
)

In [13]:
def get_plot_x(X,C,image_frame):
    #X = np.array([[6, 2,5],[6,4,5]])                        # Create a random point
    x = []
    G = []
    L = []
    pi = []

    for i in range(X.shape[0]):
        G.append(GenericPoint(X[i], name="X"))                   
        L.append(get_plucker_matrix(C, X[i]))
        X1 = image_frame.origin
        X2 = X1 + image_frame.dx
        X3 = X1 + image_frame.dy
        pi.append(get_plane_from_three_points(X1, X2, X3))
        x.append(to_inhomogeneous(L[i] @ pi[i]))

    x_arr = np.array(x)

    return x_arr, G, L,pi

In [14]:
x_arr_L,G_L,L_L,pi_L = get_plot_x(X,C_L,image_frameL)
x_arr_R,G_R,L_R,pi_R = get_plot_x(X,C_R,image_frameR)

xL = image_planeL.frame_pts()
xR = image_planeR.frame_pts()

In [15]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")
ax.text(*C_L, "C")
world_frame.draw3d()
camera_frameL.draw3d()
camera_frameR.draw3d()
image_frameL.draw3d()
image_frameR.draw3d()
Z_L.draw3d()
Z_R.draw3d()
#image_planeL.draw3d()
#image_planeR.draw3d()
for i in range(len(G_R)):
    G_L[i].draw3d(pi_L[i], C=C_L)
    G_R[i].draw3d(pi_R[i], C=C_R)
    x_surface, y_surface, z_surface = get_plane(X,chess_dimx,chess_dimy)
    ax.plot_surface(x_surface,y_surface,z_surface)

ax.view_init(elev=30.0, azim=30.0)
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.tight_layout()
plt.show()


In [16]:
fig = plt.figure(figsize=(11,7))
axs = fig.add_subplot( projection = '3d')
axs.plot(xL[0,:],xL[1,:],xL[2,:])
axs.scatter3D(x_arr_L[:,0],x_arr_L[:,1],x_arr_L[:,2])
axs.plot(xR[0,:],xR[1,:],xR[2,:])
axs.scatter3D(x_arr_R[:,0],x_arr_R[:,1],x_arr_R[:,2])
axs.set_xlabel("X-axis")
axs.set_ylabel("Y-axis")
axs.set_zlabel("Z-axis")
plt.show()

In [17]:
def triangle_area(x1,y1,x2,y2,x3,y3):
    Triange_Area = abs((0.5)*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)))

    return Triange_Area

def rectangle_area(x1,y1,x2,y2,x3,y3):
    xDiff = abs(x1 - x2) # Using absolute value to ignore negatives
    yDiff = abs(y2 - y3)

    area = xDiff * yDiff

    return area

In [18]:
def coord_in_imageplane(xL,M):
    A = np.array([xL[0,0],xL[1,0]])
    B = np.array([xL[0,1],xL[1,1]])
    C = np.array([xL[0,2],xL[1,2]])
    D = np.array([xL[0,3],xL[1,3]])

    M = np.array([M[0], M[1]]) 
    areaAMD = triangle_area(A[0],A[1],M[0],M[1],D[0],D[1])
    areaDPC = triangle_area(D[0],D[1],M[0],M[1],C[0],C[1])
    areaCMB = triangle_area(C[0],C[1],M[0],M[1],B[0],B[1])
    areaMBA = triangle_area(M[0],M[1],B[0],B[1],A[0],A[1])

    triangle_sum = areaAMD + areaDPC + areaCMB + areaMBA

    area_rectangle = rectangle_area(A[0],A[1],B[0],B[1],C[0],C[1])
    if(round(triangle_sum,2) > area_rectangle):
        #print("The point" + str(M) + " is out of the rectangle.")
        point = 1
    else:
        point = 0
    return point

## Multiple chessboards

### Get the homographies of the different chessboards

In [19]:
def repeat_func(xrange,yrange,zrange):

    translst= []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    len(translst)
    #print(translst)

    # chessboard print
    chess_pts = []
    world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0],[np.pi/3,0,-np.pi/2],[np.pi/3,0,np.pi/2],[0,np.pi/3,np.pi/2],[0,-np.pi/3,np.pi/2]]

    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X = np.linspace(0, pattern_size_x,pattern_size_x + 1)
    Y = np.linspace(0, pattern_size_y,pattern_size_y + 1)

    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X, Y)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),3),np.float32)
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:] = world_pts_

    world_pts_arr = world_pts_arr.reshape(world_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1),3)
    
    x_lst_R = []
    x_lst_L = []
    for i in range(chess_pts_arr.shape[0]):
        x_arr_R, X_arr_R, E_R,K_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F*(1/pixel_width))
        x_arr_L, X_arr_L, E_L,K_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax = THETA_X_L,thetay = THETA_Y_L, thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F*(1/pixel_width))
        
        x_lst_R.append(x_arr_R)
        x_lst_L.append(x_arr_L)

    x_zhang_R = np.array(x_lst_R,np.float32)
    x_zhang_L = np.array(x_lst_L,np.float32)

    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)


    return world_pts_arr,x_zhang_R,x_zhang_L


In [20]:
def repeat_func_no_angle(xrange,yrange,zrange):

    translst= []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    len(translst)
    #print(translst)

    # chessboard print
    chess_pts = []
    world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0]]

    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X = np.linspace(0, pattern_size_x,pattern_size_x + 1)
    Y = np.linspace(0, pattern_size_y,pattern_size_y + 1)

    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X, Y)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),3),np.float32)
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:] = world_pts_

    world_pts_arr = world_pts_arr.reshape(world_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1),3)
    
    x_lst_R = []
    x_lst_L = []
    for i in range(chess_pts_arr.shape[0]):
        x_arr_R, X_arr_R, E_R,K_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F*(1/pixel_width))
        x_arr_L, X_arr_L, E_L,K_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax = THETA_X_L,thetay = THETA_Y_L, thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F*(1/pixel_width))
        
        x_lst_R.append(x_arr_R)
        x_lst_L.append(x_arr_L)

    x_zhang_R = np.array(x_lst_R,np.float32)
    x_zhang_L = np.array(x_lst_L,np.float32)

    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)


    return world_pts_arr,x_zhang_R,x_zhang_L


In [21]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat

In [24]:
xrange_0 = np.linspace(-3000,600,4)
yrange_0 = np.linspace(-350,350,4)
zrange_0 = ([000])


world_pts_arr_2,x_zhang_R,x_zhang_L = repeat_func_no_angle(xrange_0,yrange_0,zrange_0)

fig = plt.figure(figsize=(20, 11))
fig.suptitle('Left and Right image coordinates', fontsize=16)
ax_L = fig.add_subplot(121)
ax_L.plot(x_zhang_L[:,:,0],x_zhang_L[:,:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
#ax_L.set_title("reprojection error: " + str(err_list_L))
ax_L.set_xlabel("X axis")
ax_L.set_ylabel("Y axis")
ax_L.grid()
#ax_L.legend()
ax_L.set_xlim([0,image_size[0]])
ax_L.set_ylim([0,image_size[1]])



ax_R = fig.add_subplot(122)
ax_R.plot(x_zhang_R[:,:,0],x_zhang_R[:,:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
#ax_R.set_title("reprojection error: " + str(err_list_R))
ax_R.set_xlabel("X axis")
ax_R.set_ylabel("Y axis")
ax_R.grid()
#ax_R.legend()
ax_R.set_xlim([0,image_size[0]])
ax_R.set_ylim([0,image_size[1]])



World points:  (16, 1, 63, 3)
Image points:  (16, 63, 2)


(0.0, 1216.0)

In [ ]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []


xrange_0 = np.linspace(-6000,6000,1)
yrange_0 = np.linspace(-4000,4000,1)
zrange_0 = ([20000])

xrange_1 = np.linspace(-6000,6000,2)
yrange_1 = np.linspace(-4000,4000,2)
zrange_1 = ([20000])

xrange_2 = np.linspace(-6000,6000,4)
yrange_2 = np.linspace(-4000,4000,4)
zrange_2 = ([20000])

xrange_3 = np.linspace(-6000,6000,6)
yrange_3 = np.linspace(-4000,4000,6)
zrange_3 = ([20000])

xrange_4 = np.linspace(-6000,6000,8)
yrange_4 = np.linspace(-4000,4000,8)
zrange_4 = ([20000])

xrange_5 = np.linspace(-6000,6000,10)
yrange_5 = np.linspace(-4000,4000,10)
zrange_5 = ([20000])


x_range = ([xrange_5])
y_range = ([yrange_5])
z_range = ([zrange_5])

for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L = repeat_func(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])

    print(ret_R_list)
    print(ret_L_list)
    print(retS_list)

In [67]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []


xrange_0 = np.linspace(-800,1200,1)
yrange_0 = np.linspace(-800,800,1)
zrange_0 = ([4000])

xrange_1 = np.linspace(-800,1200,2)
yrange_1 = np.linspace(-800,800,2)
zrange_1 = ([4000])

xrange_2 = np.linspace(-800,1200,4)
yrange_2 = np.linspace(-800,800,4)
zrange_2 = ([4000])

xrange_3 = np.linspace(-800,1200,6)
yrange_3 = np.linspace(-800,800,6)
zrange_3 = ([4000])

xrange_4 = np.linspace(-800,1200,8)
yrange_4 = np.linspace(-800,800,8)
zrange_4 = ([4000])

xrange_5 = np.linspace(-800,1200,10)
yrange_5 = np.linspace(-800,800,10)
zrange_5 = ([4000])

xrange_6 = np.linspace(-800,1200,12)
yrange_6 = np.linspace(-800,800,12)
zrange_6 = ([4000])

xrange_7 = np.linspace(-800,1200,14)
yrange_7 = np.linspace(-800,800,14)
zrange_7 = ([4000])

xrange_8 = np.linspace(-800,1200,16)
yrange_8 = np.linspace(-800,800,16)
zrange_8 = ([4000])


x_range = ([xrange_0,xrange_1,xrange_2,xrange_3,xrange_4,xrange_5,xrange_6,xrange_7,xrange_8])
y_range = ([yrange_0,yrange_1,yrange_2,yrange_3,yrange_4,yrange_5,yrange_6,yrange_7,yrange_8])
z_range = ([zrange_0,zrange_1,zrange_2,zrange_3,zrange_4,zrange_5,zrange_6,zrange_7,zrange_8])

for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L = repeat_func_no_angle(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])

    print(ret_R_list)
    print(ret_L_list)
    print(retS_list)

World points:  (1, 1, 63, 3)
Image points:  (1, 63, 2)
[[2.652732596350878e-06]]
[[7.47555461109732e-06]]
[[1.554291081677651e-05]]
World points:  (4, 1, 63, 3)
Image points:  (4, 63, 2)
[[2.652732596350878e-06], [2.112861979909463e-05]]
[[7.47555461109732e-06], [2.175191418667409e-05]]
[[1.554291081677651e-05], [0.003164093275621376]]
World points:  (16, 1, 63, 3)
Image points:  (16, 63, 2)
[[2.652732596350878e-06], [2.112861979909463e-05], [2.0442369425921842e-05]]
[[7.47555461109732e-06], [2.175191418667409e-05], [2.4400752403195222e-05]]
[[1.554291081677651e-05], [0.003164093275621376], [0.0013687485213569708]]
World points:  (36, 1, 63, 3)
Image points:  (36, 63, 2)
[[2.652732596350878e-06], [2.112861979909463e-05], [2.0442369425921842e-05], [4.2367848225094516e-05]]
[[7.47555461109732e-06], [2.175191418667409e-05], [2.4400752403195222e-05], [2.5958669613540595e-05]]
[[1.554291081677651e-05], [0.003164093275621376], [0.0013687485213569708], [0.0019967440448552367]]
World points:  

In [49]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []

xrange_0 = np.linspace(-100,600,1)
yrange_0 = np.linspace(-350,350,1)
zrange_0 = ([2000])

xrange_1 = np.linspace(-100,600,2)
yrange_1 = np.linspace(-350,350,2)
zrange_1 = ([2000])

xrange_2 = np.linspace(-100,600,4)
yrange_2 = np.linspace(-350,350,4)
zrange_2 = ([2000])

xrange_3 = np.linspace(-100,600,6)
yrange_3 = np.linspace(-350,350,6)
zrange_3 = ([2000])

xrange_4 = np.linspace(-100,600,8)
yrange_4 = np.linspace(-350,350,8)
zrange_4 = ([2000])

xrange_5 = np.linspace(-100,600,10)
yrange_5 = np.linspace(-350,350,10)
zrange_5 = ([2000])

x_range = ([xrange_0,xrange_1,xrange_2,xrange_3,xrange_4,xrange_5])
y_range = ([yrange_0,yrange_1,yrange_2,yrange_3,yrange_4,yrange_5])
z_range = ([zrange_0,zrange_1,zrange_2,zrange_3,zrange_4,zrange_5])

for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L = repeat_func(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])

    print(ret_R_list)
    print(ret_L_list)
    print(retS_list)

World points:  (5, 1, 63, 3)
Image points:  (5, 63, 2)
[[4.94937164235365e-06]]
[[1.728677523077448e-05]]
[[1.7045903659711926e-05]]
World points:  (20, 1, 63, 3)
Image points:  (20, 63, 2)
[[4.94937164235365e-06], [2.8996588676836916e-05]]
[[1.728677523077448e-05], [3.232156418112293e-05]]
[[1.7045903659711926e-05], [5.214576319544448e-05]]
World points:  (80, 1, 63, 3)
Image points:  (80, 63, 2)
[[4.94937164235365e-06], [2.8996588676836916e-05], [2.4398932972862228e-05]]
[[1.728677523077448e-05], [3.232156418112293e-05], [3.446720525238006e-05]]
[[1.7045903659711926e-05], [5.214576319544448e-05], [3.344784485053274e-05]]
World points:  (180, 1, 63, 3)
Image points:  (180, 63, 2)
[[4.94937164235365e-06], [2.8996588676836916e-05], [2.4398932972862228e-05], [2.3344647432335383e-05]]
[[1.728677523077448e-05], [3.232156418112293e-05], [3.446720525238006e-05], [3.399383665159746e-05]]
[[1.7045903659711926e-05], [5.214576319544448e-05], [3.344784485053274e-05], [3.058896361321599e-05]]
Worl

In [22]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []


xrange_0 = np.linspace(-800,1200,1)
yrange_0 = np.linspace(-800,800,1)
zrange_0 = ([4000])

xrange_1 = np.linspace(-800,1200,2)
yrange_1 = np.linspace(-800,800,2)
zrange_1 = ([4000])

xrange_2 = np.linspace(-800,1200,4)
yrange_2 = np.linspace(-800,800,4)
zrange_2 = ([4000])

xrange_3 = np.linspace(-800,1200,6)
yrange_3 = np.linspace(-800,800,6)
zrange_3 = ([4000])

xrange_4 = np.linspace(-800,1200,8)
yrange_4 = np.linspace(-800,800,8)
zrange_4 = ([4000])

xrange_5 = np.linspace(-800,1200,10)
yrange_5 = np.linspace(-800,800,10)
zrange_5 = ([4000])

xrange_6 = np.linspace(-800,1200,12)
yrange_6 = np.linspace(-800,800,12)
zrange_6 = ([4000])

xrange_7 = np.linspace(-800,1200,14)
yrange_7 = np.linspace(-800,800,14)
zrange_7 = ([4000])

xrange_8 = np.linspace(-800,1200,16)
yrange_8 = np.linspace(-800,800,16)
zrange_8 = ([4000])


x_range = ([xrange_0,xrange_1,xrange_2,xrange_3,xrange_4,xrange_5,xrange_6,xrange_7,xrange_8])
y_range = ([yrange_0,yrange_1,yrange_2,yrange_3,yrange_4,yrange_5,yrange_6,yrange_7,yrange_8])
z_range = ([zrange_0,zrange_1,zrange_2,zrange_3,zrange_4,zrange_5,zrange_6,zrange_7,zrange_8])

for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L = repeat_func(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])

    print(ret_R_list)
    print(ret_L_list)
    print(retS_list)

World points:  (5, 1, 63, 3)
Image points:  (5, 63, 2)
[[2.384270910265176e-06]]
[[9.386274520290202e-06]]
[[3.6357221997741565e-05]]
World points:  (20, 1, 63, 3)
Image points:  (20, 63, 2)
[[2.384270910265176e-06], [2.7848762294205886e-05]]
[[9.386274520290202e-06], [2.7564733805145732e-05]]
[[3.6357221997741565e-05], [0.00012362859641961232]]
World points:  (80, 1, 63, 3)
Image points:  (80, 63, 2)
[[2.384270910265176e-06], [2.7848762294205886e-05], [2.4615667263683825e-05]]
[[9.386274520290202e-06], [2.7564733805145732e-05], [2.8211943902951292e-05]]
[[3.6357221997741565e-05], [0.00012362859641961232], [7.856812067171512e-05]]
World points:  (180, 1, 63, 3)
Image points:  (180, 63, 2)


In [ ]:
x_stereo_4m = [[3.7758901173734655e-05], [0.00026299191932619223], [6.0374409680135805e-05],[0.0001075]]

single_x = [5,20,80,180]

fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot()
ax.plot(single_x,x_stereo_4m,'g-*',label = "4 meters")
ax.grid()
ax.set_ylabel("Reprojection error( in pixel)")
ax.set_xlabel("number of chessboards")
ax.set_title("Stereo camera calibration error")
ax.legend()